In [ ]:
pip install Prophet 

In [ ]:
pip install TA-Lib

In [ ]:
pip install pandas_ta

In [15]:
from prophet import Prophet
import pandas as pd
import datetime
import yfinance as yf
import pandas_ta as ta
import plotly.express as px
import plotly.graph_objs as go
from prophet.diagnostics import cross_validation
from prophet.diagnostics import performance_metrics

In [2]:
# Baixar dados históricos usando yfinance
symbol = 'BBAS3.SA'  # Símbolo da ação desejada aqui 
end_date = datetime.datetime.now()
start_date = end_date - datetime.timedelta(days = 365)
interval = '1h'
stock_data = yf.download(symbol, start=start_date, end=end_date, interval=interval)

display(stock_data)

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
2022-12-20 10:00:00-03:00,32.939999,33.900002,32.630001,33.540001,33.540001,2381900
2022-12-20 11:00:00-03:00,33.540001,34.049999,33.529999,33.779999,33.779999,2247600
2022-12-20 12:00:00-03:00,33.790001,33.990002,33.639999,33.950001,33.950001,1374000
2022-12-20 13:00:00-03:00,33.959999,33.970001,33.580002,33.820000,33.820000,1190400
2022-12-20 14:00:00-03:00,33.810001,33.919998,33.779999,33.790001,33.790001,1051400
...,...,...,...,...,...,...
2023-12-19 13:00:00-03:00,54.580002,54.680000,54.450001,54.599998,54.599998,745500
2023-12-19 14:00:00-03:00,54.599998,54.770000,54.570000,54.700001,54.700001,510300
2023-12-19 15:00:00-03:00,54.700001,54.900002,54.540001,54.660000,54.660000,1083700
2023-12-19 16:00:00-03:00,54.660000,54.860001,54.599998,54.720001,54.720001,1015500


In [8]:
data = stock_data[['Adj Close']].copy()
data.index = pd.to_datetime(data.index)
data['ds'] = data.index.strftime('%Y-%m-%d %H:%M:%S')

# use technical analyses using 21 one days and append to our dataset
data.ta.ema(close='Adj Close', length=21, append=True)

# Drop empty values
data.dropna(inplace=True)

In [9]:
#Plot
fig = px.line(data, x= 'ds', y='Adj Close')

fig.update_xaxes(
    rangeslider_visible=True,
    rangeselector=dict(
        buttons=list([
            dict(count=1, label="1m", step="month", stepmode="backward"),
            dict(count=6, label="6m", step="month", stepmode="backward"),
            dict(count=1, label="YTD", step="year", stepmode="todate"),
            dict(count=1, label="1y", step="year", stepmode="backward"),
            dict(step="all")
        ])
    )
)

In [ ]:
pip install prophet --upgrade

In [ ]:
pip install pystan --upgrade

In [ ]:
pip install pandas numpy Cython cmdstanpy

In [10]:
# Add all dataset as training model
df_train = data[[ 'ds', 'Adj Close','EMA_21']]

df_train = df_train.rename(columns={"Adj Close": "y"})
#display(df_train)

In [27]:
conda install -c conda-forge prophet

^C

Note: you may need to restart the kernel to use updated packages.


In [9]:
pip install pystan==2.19.1.1

Note: you may need to restart the kernel to use updated packages.


In [12]:
# Fit model
m = Prophet(daily_seasonality=True)

# Train 
m.fit(df_train)

# Get dates 30 days in the future
future = m.make_future_dataframe(periods=40)

# Drop weekends
future['day'] =  future['ds'].dt.weekday
future = future[future['day'] <=4]

# Predict dates
forecast = m.predict(future)

00:56:52 - cmdstanpy - INFO - Chain [1] start processing
00:56:55 - cmdstanpy - INFO - Chain [1] done processing


In [13]:

# Plot linear regression result
fig = go.Figure([
    go.Scatter(x=df_train['ds'], y=df_train['y'], name='Actual', mode='lines'),
    go.Scatter(x=forecast['ds'], y=forecast['yhat'], name='Predicted', mode='lines'),
    go.Scatter(x=forecast['ds'], y=df_train['EMA_21'], name='EMA', mode='lines')
])

fig.update_xaxes(
    rangeslider_visible=True,
    rangeselector=dict(
        buttons=list([
            dict(count=1, label="1d", step="day", stepmode="backward"),
            dict(count=1, label="1m", step="month", stepmode="backward"),
            dict(count=6, label="6m", step="month", stepmode="backward"),
            dict(count=1, label="YTD", step="year", stepmode="todate"),
            dict(count=1, label="1y", step="year", stepmode="backward"),
            dict(step="all")
        ])
    )
)
fig.show()

In [16]:

# Get all predictions
pred_df = forecast[forecast['ds'] > datetime.today()][['ds','yhat']]

# Reset Index
pred_df.reset_index(inplace=True)

# Drop index column
pred_df.drop(labels='index', axis=1,inplace=True)

# Rename columns
pred_df.rename(columns={'ds': 'date', 'yhat': 'predicted price'}, inplace=True)

# Show first elements
pred_df

AttributeError: module 'datetime' has no attribute 'today'

In [17]:
# execute cross validation

# reference
# https://facebook.github.io/prophet/docs/diagnostics.html#:~:text=Cross%20validation,up%20to%20that%20cutoff%20point.

df_cv = cross_validation(m, initial='720 days', period='30 days', horizon = '365 days')

# visualize data
print(df_cv.head())

# measure performance
df_p = performance_metrics(df_cv)
print(df_p.head())

ValueError: Less data than horizon.